In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
from pathlib import Path
import numpy as np
import pytorch_lightning as pl

import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
from utils.topo_ds import *
import seaborn as sns 
from utils.pershom import pers2fn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, GradientAccumulationScheduler
from pl_modules.pl_roberta import LitRelRoberta
from ruamel.yaml import YAML

from sklearn.metrics import precision_recall_fscore_support, mean_absolute_error
from utils.sim_rel import compare_topo_models, topo_model, compare_CKA_models
from functools import partial

from utils.export_multilingual_results import process_df
from utils.multilingual_amazon_anchors import MultilingualAmazonAnchors
from typing import *

from modules.stitching_module import StitchingModule

from datasets import load_dataset, ClassLabel, Dataset, DatasetDict

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Tensorboard extension (for visualization purposes later)
%load_ext tensorboard

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = Path("./data")
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = Path("./saved_models/rel_multi_topo")
CHECKPOINT_PATH_VANILLA = Path("./saved_models/rel_multi_vanilla")
RESULT_PATH = Path("./results/rel_multi_topo")
FIG_PATH =  Path("./Visualization")

PROJECT_ROOT = Path("./")

pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Global seed set to 42


Device: cuda:0


# Data

In [3]:
fine_grained: bool = True
target_key: str = "class"
data_key: str = "content"
anchor_dataset_name: str = "amazon_translated"  
ALL_LANGS = ("en", "fr")
num_anchors: int = 768
train_perc: float = 0.01

In [4]:
def get_dataset(lang: str, split: str, perc: float, fine_grained: bool):
    pl.seed_everything(42)
    assert 0 < perc <= 1
    
    # Amazon removed the public dataset :(
    # dataset = load_dataset("amazon_reviews_multi", lang)[split]
    
    # Using files from https://www.kaggle.com/datasets/mexwell/amazon-reviews-multi
    # Read CSV files into pandas DataFrames
    
    split_df = pd.read_csv(DATASET_PATH / f"multi/{split}.csv", engine='python')
    try:
        split_df = pd.read_csv(DATASET_PATH / f"multi/{split}.csv", engine='python')
    except:
        raise Exception("Download files from https://www.kaggle.com/datasets/mexwell/amazon-reviews-multi and add them to ./data/multi/")
    
    # Filter by language
    lang_split_df = split_df[split_df['language'] == lang]
    
    # Create Hugging Face datasets
    dataset = Dataset.from_pandas(lang_split_df)
    dataset = dataset.remove_columns(['Unnamed: 0', '__index_level_0__'])

    if not fine_grained:
        dataset = dataset.filter(lambda sample: sample["stars"] != 3)

    # Select a random subset
    indices = list(range(len(dataset)))
    random.shuffle(indices)
    indices = indices[: int(len(indices) * perc)]
    dataset = dataset.select(indices)

    def clean_sample(sample):
        title: str = sample["review_title"].strip('"').strip(".").strip()
        body: str = sample["review_body"].strip('"').strip(".").strip()

        if body.lower().startswith(title.lower()):
            title = ""

        if len(title) > 0 and title[-1].isalpha():
            title = f"{title}."

        sample["content"] = f"{title} {body}".lstrip(".").strip()
        if fine_grained:
            sample[target_key] = str(sample["stars"] - 1)
        else:
            sample[target_key] = sample["stars"] > 3
        return sample

    dataset = dataset.map(clean_sample)
    dataset = dataset.cast_column(
        target_key,
        ClassLabel(num_classes=5 if fine_grained else 2, names=list(map(str, range(5) if fine_grained else (0, 1)))),
    )

    return dataset

def _amazon_translated_get_samples(lang: str, sample_idxs):
    anchor_dataset = MultilingualAmazonAnchors(split="train", language=lang)
    anchors = []
    for anchor_idx in sample_idxs:
        anchor = anchor_dataset[anchor_idx]
        anchor[data_key] = anchor["data"]
        anchors.append(anchor)
    return anchors

In [5]:
train_datasets = {
    lang: get_dataset(lang=lang, split="train", perc=train_perc, fine_grained=fine_grained) for lang in ALL_LANGS
    }

test_datasets = {
    lang: get_dataset(lang=lang, split="test", perc=1, fine_grained=fine_grained) for lang in ALL_LANGS
    }

val_datasets = {
    lang: get_dataset(lang=lang, split="validation", perc=1, fine_grained=fine_grained) for lang in ALL_LANGS
    }

num_labels = len(np.unique(train_datasets["en"][target_key], return_counts=True)[0])
print("Num labels:", num_labels)

Global seed set to 42


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Global seed set to 42


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

...

Casting the dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Global seed set to 42


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Num labels: 5


In [6]:
assert len(set(frozenset(train_dataset.features.keys()) for train_dataset in train_datasets.values())) == 1
class2idx = train_datasets["en"].features[target_key].str2int

train_datasets["en"].features

{'review_id': Value(dtype='string', id=None),
 'product_id': Value(dtype='string', id=None),
 'reviewer_id': Value(dtype='string', id=None),
 'stars': Value(dtype='int64', id=None),
 'review_body': Value(dtype='string', id=None),
 'review_title': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'product_category': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None),
 'class': ClassLabel(names=['0', '1', '2', '3', '4'], id=None)}

Get pararel anchors

In [7]:
anchor_dataset2num_samples = 1000
anchor_dataset2first_anchors = [
        776,
        507,
        895,
        922,
        33,
        483,
        85,
        750,
        354,
        523,
        184,
        809,
        418,
        615,
        682,
        501,
        760,
        49,
        732,
        336,
    ]


assert num_anchors <= anchor_dataset2num_samples

pl.seed_everything(42)
anchor_idxs = list(range(anchor_dataset2num_samples))
random.shuffle(anchor_idxs)
anchor_idxs = anchor_idxs[:num_anchors]

assert anchor_idxs[:20] == anchor_dataset2first_anchors  # better safe than sorry
lang2anchors = {
    lang: _amazon_translated_get_samples(lang=lang, sample_idxs=anchor_idxs) for lang in ALL_LANGS
}

Global seed set to 42


This is how we can handdle automatically the tokenizer

In [8]:
def collate_fn(batch, tokenizer, cls=True):
    data = []
    labels = []
    
    for x, y in batch:
        data.append(x)
        if cls:
            labels.append(y)

    encoding = tokenizer(
        data,
        return_tensors="pt",
        return_special_tokens_mask=True,
        truncation=True,
        max_length=512,
        padding=True,
    )
    del encoding["special_tokens_mask"]

    if cls:
        result = (encoding, torch.tensor(labels))
    else:
        result = encoding

    return  result

In [9]:
def multi_draw_collate_fn(batch, tokenizer, cls=True):
    data = []
    labels = []    
    for x, y in batch:
        data += x 
        if cls:
            labels += [y]*len(x)

    encoding = tokenizer(
        data,
        return_tensors="pt",
        return_special_tokens_mask=True,
        truncation=True,
        max_length=512,
        padding=True,
    )
    del encoding["special_tokens_mask"]

    if cls:
        result = (encoding, torch.tensor(labels))
    else:
        result = encoding

    return  result

In [10]:
lang2transformer_name = {
    "en": "roberta-base",
    "fr": "ClassCat/roberta-base-french",
    #"es": "PlanTL-GOB-ES/roberta-base-bne",
    #"ja": "nlp-waseda/roberta-base-japanese",
}
assert set(lang2transformer_name.keys()) == set(ALL_LANGS)

In [11]:
train_lang2dataloader = {}
test_lang2dataloader = {}
val_lang2dataloader = {}
anchors_lang2dataloader = {}

for lang in ALL_LANGS:
    transformer_name = lang2transformer_name[lang]
    print(transformer_name)
    lang_tokenizer = AutoTokenizer.from_pretrained(transformer_name)
    
    aux_train_ds = DictDataset(train_datasets[lang], data_key, target_key)    
    train_lang2dataloader[lang] = DataLoader(aux_train_ds,
                                       num_workers=4,
                                       collate_fn=partial(collate_fn, tokenizer=lang_tokenizer),
                                       pin_memory=True,
                                       #persistent_workers= True,
                                       batch_sampler=ClassAccumulationSampler(aux_train_ds,
                                                       batch_size=16,
                                                       drop_last=True,
                                                       accumulation=num_labels,
                                                       indv=True,
                                                       main_random=True
                                                      )
                                            )
                                  
    
    aux_test_ds = DictDataset(test_datasets[lang], data_key, target_key)
    ds_test_multi = IntraLabelMultiDraw(aux_test_ds, 16)
    test_lang2dataloader[lang] = (DataLoader(aux_test_ds,
                                       num_workers=4,
                                       collate_fn=partial(collate_fn, tokenizer=lang_tokenizer),
                                       batch_size=32,
                                       pin_memory=True
                                       ),
                                  DataLoader(ds_test_multi,
                                       num_workers=4,
                                       collate_fn=partial(multi_draw_collate_fn, tokenizer=lang_tokenizer, cls=False),
                                       batch_size=1,
                                       pin_memory=True
                                       )
                                 )
    
    
    aux_val_ds = DictDataset(val_datasets[lang], data_key, target_key)
    
    
    val_lang2dataloader[lang] = DataLoader(aux_val_ds,
                                       num_workers=4,
                                       collate_fn=partial(collate_fn, tokenizer=lang_tokenizer),
                                       batch_size=32,
                                       pin_memory=True
                                       )
    
    aux_anc_ds = DictDataset(lang2anchors[lang], data_key, target_key)
     
    anchors_lang2dataloader[lang] = DataLoader(aux_anc_ds,
                                       num_workers=4,
                                       pin_memory=True,
                                       collate_fn=partial(collate_fn, tokenizer=lang_tokenizer, cls=False),
                                       batch_size=16,
                                       )

roberta-base
ClassCat/roberta-base-french


# Train

In [12]:
from pl_modules.pl_topo_roberta import LitTopoRelRoberta

In [13]:
def test_model(model, dataloader, title=""):
    preds = []
    model.to(device)
    model.eval()
    with torch.no_grad():
        batch_idx = 0
        for batch, _ in tqdm(dataloader, position=0, leave=True, desc="Computing"+title):
            batch.to(device)
            batch_latents = model(batch_idx=batch_idx, **batch)["prediction"].argmax(-1)
            preds.append(batch_latents)
            batch_idx = 1

    preds = torch.cat(preds, dim=0).detach().cpu().numpy()
    test_y = np.array(test_datasets["en"][target_key])

    precision, recall, fscore, _ = precision_recall_fscore_support(test_y, preds, average="weighted")
    mae = mean_absolute_error(y_true=test_y, y_pred=preds)
    acc = (preds == test_y).mean()
    return precision, recall, acc, fscore, mae

In [14]:
torch.autograd.set_detect_anomaly(True)

EPOCHS = 40 if fine_grained else 3


def train_network(lang, mode="relative", seed=24, test=False, topo=("pre", "L_2", 7, 0.1, "L_1")):
    
    # Create a PyTorch Lightning trainer with the generation callback
    aux = 'fine_grained' if fine_grained else 'coarse_grained'
    if topo is None:
        aux = f"{aux}/vanilla_linear_{lang}_{mode}_seed{seed}"
    else:
        print(topo[2])

        if  mode == "relative":
            aux = f"{aux}/topo_linear_{lang}_{mode}_{topo[0]}_seed{seed}"
        else:
            aux = f"{aux}/topo_linear_{lang}_{mode}_seed{seed}"
        
    title = CHECKPOINT_PATH /  aux
    trainer = pl.Trainer(default_root_dir=title, 
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         gradient_clip_val=1,
                         accumulate_grad_batches=num_labels+1,
                         max_epochs=EPOCHS, 
                         reload_dataloaders_every_n_epochs=1,
                         callbacks=[ModelCheckpoint(save_weights_only=True,
                                                   mode="max", monitor="val_acc",
                                                   filename="best"
                                                   ),
                                    ModelCheckpoint(save_weights_only=True,
                                                    filename="full"
                                                   ),
                                    LearningRateMonitor(logging_interval='step'),
                                    ]
                         )
    
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need
    
    transformer_model = lang2transformer_name[lang]
    
    anchor_loader = None
    if mode == "relative":
        anchor_loader = anchors_lang2dataloader[lang]
    
    train_loader = train_lang2dataloader[lang]
    steps = EPOCHS*len(train_loader)
    power = 10
    freq_anchors = power*(num_labels+1)
    model = LitTopoRelRoberta(
                          num_labels=num_labels,
                          transformer_model=transformer_model,
                          anchor_dataloader=anchor_loader,
                          topo_par=topo,
                          hidden_size=num_anchors,
                          # normalization_mode="batchnorm",
                          normalization_mode=None,
                          output_normalization_mode=None,
                          dropout_prob=0.1,
                          seed=seed,
                          steps=steps,
                          weight_decay=0.01, 
                          head_lr=1e-3/(num_labels),
                          encoder_lr=1.75e-4/(num_labels),
                          layer_decay=0.65,
                          scheduler_act=True,
                          freq_anchors=freq_anchors,
                          device=device,
                          fine_tune=False,
                          linear=True
                          )
    
    val_loader = val_lang2dataloader[lang]
    trainer.fit(model, train_lang2dataloader[lang], val_dataloaders=val_loader)
    
    if test:
        
        res = test_model(model.net, test_lang2dataloader[lang][0])
        with open("res.txt", "a") as f:
            f.write(f"-------------{topo}-------------\n")
            f.write(f"precision {res[0]}\n")
            f.write(f"recall {res[1]}\n")
            f.write(f"acc {res[2]}\n")
            f.write(f"fscore {res[3]}\n")
            f.write(f"mae {res[4]}\n")
            
        print("precision", res[0])
        print("recall", res[1])
        print("acc", res[2])
        print("fscore", res[3])
        print("mae", res[4])
        aux = "rel_topo" if topo is not None else "rel_vanilla"
        
        title = f"{lang.upper()} {mode}: VR " + r"$H_0$" + " pers w/ " + r"$L^2$" 
        if topo is not None:
            if topo[1]=="L_inf":
                title += " and " +r"$L^{\infty}$"
                
            if type(topo[2]) is tuple:
                title+= f" ({topo[0]}, "+ r"$\lambda={}$, ".format(topo[3]) + r"$\beta_{pre}$" +r"$={}$, ".format(topo[2][0]) +\
                    r"$\beta_{post}$" +r"$={}$)".format(topo[2][1]) 
            else:
                title+= f" ({topo[0]}, "+ r"$\lambda={}$, ".format(topo[3]) + r"$\beta={}$)".format(topo[2])
                
        if topo is not None:
            path =  f"{'fine_grained' if fine_grained else 'coarse_grained'}/full_topo_dist_linear/{lang}_{mode}"
            if mode == "relative":
                path += f"_{topo[0]}"
            if type(topo[2]) is tuple:
                path += f"_{topo[2][0]}_{topo[2][1]}_seed{seed}.png"
            else:
                path += f"_{topo[2]}_seed{seed}.png"
            
        else:
            path =  f"{'fine_grained' if fine_grained else 'coarse_grained'}/full_topo_dist_linear/{lang}_{mode}_seed{seed}.png"

        topo_model(model.net, device, test_lang2dataloader[lang][1],
                 FIG_PATH / aux / path,
                 title=title,
                 pers=("L_2", "L_inf"), plot_topo=True, relative=mode=="relative");
       
    
    model.to("cpu")
    del model
    

In [18]:
rng = np.random.default_rng(100)
SEEDS = rng.integers(low=0, high=10, size=5)
run = False

if run:
    for seed in tqdm(SEEDS, leave=False, desc="seed"):
        for train_lang in tqdm(["en", "fr"], leave=False, desc="lang"):
            # Train Topo
            train_network(train_lang, mode="relative", seed=seed, test=True, topo=("both", "L_2", 3, 0.02, "L_1"), batch_norm=True)

            # Train vanilla rel without topo
            train_network(train_lang, mode="relative", seed=seed, test=True, topo=None, batch_norm=False)

            # Train robust rel without topo
            train_network(train_lang, mode="relative", seed=seed, test=True, topo=None, batch_norm=True)

            # Train absolute without topo
            train_network(train_lang, mode="absolute", seed=seed, test=True, topo=None, batch_norm=True)


# Results

## Compute

In [22]:
numeric_results = {
    "vanilla": {
        "seed": [],
        "embed_type": [],
        "enc_lang": [],
        "dec_lang": [],
        "precision": [],
        "recall": [],
        "acc": [],
        "fscore": [],
        "mae": [],
        "stitched": [],
        "CKA_pre":[],
        "CKA_post":[],
    },
     "topo": {
        "seed": [],
        "embed_type": [],
        "enc_lang": [],
        "dec_lang": [],
        "precision": [],
        "recall": [],
        "acc": [],
        "fscore": [],
        "mae": [],
        "stitched": [],
        "CKA_pre":[],
        "CKA_post":[],
    },
}

rng = np.random.default_rng(100)
SEEDS = rng.integers(low=0, high=10, size=5)

pers = "L_2"
langs= ["en", "fr"]
modes = ["vanilla"]
embeddings = [ "relative"]
check_best = False            
vrs = 1
for seed in SEEDS:
    models = {
        train_mode: {
            embedding_type: {
                    train_lang: LitTopoRelRoberta.load_from_checkpoint(
                                  CHECKPOINT_PATH / 
                                  f"{'fine_grained' if fine_grained else 'coarse_grained'}/{train_mode}_linear_{train_lang}_{embedding_type}_seed{seed}" /
                                  f"lightning_logs/version_{vrs if 'relative' in embedding_type else 0}/checkpoints/{'best' if check_best else 'full-v1'}.ckpt",
                                   linear=True)
                    

                    for train_lang in langs
                }
                for embedding_type in embeddings
        }
        for train_mode in tqdm(modes, leave=True, desc="mode")
    }
    
    for mode in modes:
        for k, embed_type in enumerate(embeddings):
            for i, enc_lang  in enumerate(langs):
                for j, dec_lang  in enumerate(langs):
                    
                    model_enc = models[mode][embed_type][enc_lang].net
                            
                    relative = "relative" in embed_type
                    if relative:
                        model_enc.anchor_dataloader = anchors_lang2dataloader[enc_lang]
                    
                    if enc_lang != dec_lang:
                        model_dec = models[mode][embed_type][dec_lang].net
                        model = StitchingModule(model_enc, model_dec)
                    else:
                        model = model_enc
                      
                        
                    # The data is paired with its encoder
                    test_loader = test_lang2dataloader[enc_lang][0]
                    title = f" {mode}_seed{seed}_{embed_type}_{enc_lang}_{dec_lang}"
                    
                    
                    precision, recall, acc, fscore, mae = test_model(model, test_loader, title)
                    
                    if j>i:
                        
                        title=f"{enc_lang.upper()}-{dec_lang.upper()} {embed_type}: VR " + r"$H_0$" + " pers w/ " + r"$L^2$"
                        
                        if mode=="topo":
                            
                                
                            with open(CHECKPOINT_PATH / 
                                        f"{'fine_grained' if fine_grained else 'coarse_grained'}/{mode}_linear_{enc_lang}_{embed_type}_seed{seed}" /
                                        f"lightning_logs/version_{vrs}/hparams.yaml") as fp:
                                topo_1 = YAML().load(fp)["topo_par"]
                            
                            if topo_1[1]=="L_inf":
                                title += " and " +r"$L^{\infty}$"
                            
                            

                            title+= f" (Model 1: {topo_1[0]}, "+ r"$\lambda={}$, ".format(topo_1[3]) + r"$\beta={}$)".format(topo_1[2])
                            
                            with open(CHECKPOINT_PATH / 
                                            f"{'fine_grained' if fine_grained else 'coarse_grained'}/{mode}_linear_{dec_lang}_{embed_type}_seed{seed}" /
                                            f"lightning_logs/version_{vrs}/hparams.yaml") as fp:
                                    topo_2 = YAML().load(fp)["topo_par"]
                                

                            title+= f" (Model 2: {topo_2[0]}, "+ r"$\lambda={}$, ".format(topo_2[3]) + r"$\beta={}$)".format(topo_2[2])
                            
                        
                        
                        if relative:
                            model_dec.anchor_dataloader = anchors_lang2dataloader[dec_lang]
                    
                        
                        if mode == "topo":
                            path =  f"{'fine_grained' if fine_grained else 'coarse_grained'}/{'best_' if check_best else ''}full_topo_dist_linear/{enc_lang}_{dec_lang}_{embed_type}"
                            if relative:
                                path += f"_{topo_1[0]}_{topo_2[0]}"
                                
                            if type(topo_1[2]) is tuple:
                                path += f"_{topo_1[2][0]}_{topo_1[2][1]}"
                            else:
                                path += f"_{topo_1[2]}"
                            
                            if type(topo_2[2]) is tuple:
                                path += f"_{topo_2[2][0]}_{topo_2[2][1]}_seed{seed}.png"
                            else:
                                path += f"_{topo_2[2]}_seed{seed}.png"
                                                   

                        else:
                            path =  f"{'fine_grained' if fine_grained else 'coarse_grained'}/{'best_' if check_best else ''}full_topo_dist_linear/{enc_lang}_{dec_lang}_{embed_type}_seed{seed}.png"


                        """compare_topo_models(model_enc, model_dec, device,
                                            dataloader1=test_lang2dataloader[enc_lang][1],
                                            dataloader2=test_lang2dataloader[dec_lang][1],
                                            save_path= FIG_PATH / "rel_vanilla" /path,
                                            title=title,
                                            pers=pers,
                                            relative=relative,
                                            )"""
                        
                        res = compare_CKA_models(model_enc, model_dec, device,
                                            dataloader1= anchors_lang2dataloader[enc_lang],
                                            dataloader2= anchors_lang2dataloader[dec_lang],
                                            relative=relative
                                            )
                        
                        if relative:
                            numeric_results[mode]["CKA_pre"].append(res[0])
                            numeric_results[mode]["CKA_post"].append(res[1])
                        else:
                            numeric_results[mode]["CKA_pre"].append(res)
                            numeric_results[mode]["CKA_post"].append(-1.)

                    elif j<i:
                        numeric_results[mode]["CKA_pre"].append(numeric_results[mode]["CKA_pre"][i + j*len(langs) + k*len(langs)**2])
                        numeric_results[mode]["CKA_post"].append(numeric_results[mode]["CKA_post"][i + j*len(langs) + k*len(langs)**2])
                    else:
                        numeric_results[mode]["CKA_pre"].append(1.)
                        if relative:
                            numeric_results[mode]["CKA_post"].append(1.)
                        else:
                            numeric_results[mode]["CKA_post"].append(-1.)
                    
                    if not relative:
                        numeric_results[mode]["embed_type"].append(embed_type)
                    else:
                        numeric_results[mode]["embed_type"].append("relative")
                    numeric_results[mode]["enc_lang"].append(enc_lang)
                    numeric_results[mode]["dec_lang"].append(dec_lang)
                    numeric_results[mode]["precision"].append(precision)
                    numeric_results[mode]["recall"].append(recall)
                    numeric_results[mode]["acc"].append(acc)
                    numeric_results[mode]["fscore"].append(fscore)
                    numeric_results[mode]["stitched"].append(enc_lang != dec_lang)
                    numeric_results[mode]["mae"].append(mae)
                    numeric_results[mode]["seed"].append(seed)
    
    for mode in modes:
        for embed_type in embeddings:
            for enc_lang in langs:
                   del models[mode][embed_type][enc_lang]

mode:   0%|                                                                               | 0/1 [00:00<?, ?it/s]Global seed set to 7
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Global seed set to 7
Some weights of the model checkpoint at ClassCat/roberta-base-french were not used 

Computing vanilla_seed5_relative_en_fr: 100%|█████████████████████████████████| 157/157 [00:16<00:00,  9.32it/s]
Computing: 48it [00:09,  5.30it/s]
mode:   0%|                                                                               | 0/1 [00:00<?, ?it/s]Global seed set to 0
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

In [23]:
df = pd.DataFrame(numeric_results["vanilla"])
df

,seed,embed_type,enc_lang,dec_lang,precision,recall,acc,fscore,mae,stitched,CKA_pre,CKA_post
0,7,relative,en,en,0.604430,0.6060,0.6060,0.603921,0.4524,False,1.000000,1.000000
1,7,relative,en,fr,0.610564,0.6092,0.6092,0.603179,0.4506,True,-0.006059,-0.004059
2,7,relative,fr,en,0.533070,0.5106,0.5106,0.515025,0.5664,True,-0.006059,-0.004059
3,7,relative,fr,fr,0.531707,0.5072,0.5072,0.514497,0.5686,False,1.000000,1.000000
4,8,relative,en,en,0.599674,0.6008,0.6008,0.593888,0.4622,False,1.000000,1.000000
5,8,relative,en,fr,0.604179,0.5962,0.5962,0.589419,0.4632,True,0.004621,0.016551
6,8,relative,fr,en,0.535115,0.5182,0.5182,0.521533,0.5586,True,-0.006059,-0.004059
7,8,relative,fr,fr,0.534992,0.5104,0.5104,0.516393,0.5644,False,1.000000,1.000000
8,1,relative,en,en,0.618261,0.6150,0.6150,0.613856,0.4422,False,1.000000,1.000000
9,1,relative,en,fr,0.626490,0.6168,0.6168,0.614034,0.4376,True,0.002311,0.003321


In [24]:
for mode in ["vanilla_redo_4"]:
    aux = f"nlp_multilingual-stitching-amazon-{'fine_grained' if fine_grained else 'coarse_grained'}-{mode}-{train_perc}"
    
    if check_best:
        aux+="-best"
    aux += ".tsv"
    
    if "topo" in mode:
        df = pd.DataFrame(numeric_results["topo"])
    else:
        df = pd.DataFrame(numeric_results["vanilla"])
    
    
#     df_2 = pd.read_csv(
#         RESULT_PATH / aux,
#         sep='\t')
    
#     df = pd.concat([df_2, df])
    
    
    df.to_csv(
        RESULT_PATH / aux,
        sep="\t",
        index=False
    )



## View

In [25]:
fine_grained: bool = True
check_best = False

df_fine, tex_fine = process_df(fine_grained=fine_grained, mode="vanilla_redo_4", path=RESULT_PATH, train_perc=train_perc, best=check_best)

print(tex_fine)

df_fine

\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{
\begin{tabular}{clcccccc}
\toprule
   &    & \multicolumn{3}{c}{Absolute} & \multicolumn{3}{c}{Relative} \\
 \cmidrule(lr){3-5} 
 \cmidrule(lr){6-8} 
 Decoder & Encoder & Acc & FScore & MAE & Acc & FScore & MAE \\
\midrule
\multirow{2}{*}{en} & en &   $59.26 \pm 0.66$ &  $58.27 \pm 0.83$ &    $49.52 \pm 0.89$ &  $60.84 \pm 0.64$ &  $60.30 \pm 0.72$ &  $45.35 \pm 0.74$ \\
   & fr &  $24.28 \pm 10.11$ &  $22.27 \pm 8.86$ &  $139.27 \pm 35.32$ &  $49.92 \pm 1.51$ &  $50.13 \pm 1.60$ &  $57.56 \pm 1.60$ \\
&\\
\multirow{2}{*}{fr} & en &   $24.96 \pm 9.27$ &  $23.19 \pm 8.12$ &  $132.35 \pm 24.01$ &  $60.74 \pm 0.88$ &  $60.18 \pm 1.14$ &  $45.19 \pm 1.16$ \\
   & fr &   $49.26 \pm 1.04$ &  $48.74 \pm 0.73$ &    $63.89 \pm 1.50$ &  $50.31 \pm 0.88$ &  $50.95 \pm 0.82$ &  $57.08 \pm 1.22$ \\
\bottomrule
\end{tabular}
}
\caption{Fine grained: vanilla_redo_4}
\label{tab:multilingual-vanilla_redo_4-fine-grained}
\end{table}


Absolute                                    Relative        \
                     acc        fscore           mae             acc         
                    mean    std   mean   std    mean    std     mean   std   
Decoder Encoder                                                              
en      en         59.26   0.66  58.27  0.83   49.52   0.89    60.84  0.64   
        fr         24.28  10.11  22.27  8.86  139.27  35.32    49.92  1.51   
fr      en         24.96   9.27  23.19  8.12  132.35  24.01    60.74  0.88   
        fr         49.26   1.04  48.74  0.73   63.89   1.50    50.31  0.88   

                                           
                fscore          mae        
                  mean   std   mean   std  
Decoder Encoder                            
en      en       60.30  0.72  45.35  0.74  
        fr       50.13  1.60  57.56  1.60  
fr      en       60.18  1.14  45.19  1.16  
        fr       50.95  0.82  57.08  1.22

In [20]:
fine_grained: bool = True

df_fine, tex_fine = process_df(fine_grained=fine_grained, mode="topo_redo", path=RESULT_PATH, train_perc=train_perc, best=check_best)

print(tex_fine)

df_fine

\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{
\begin{tabular}{clcccccc}
\toprule
   &    & \multicolumn{3}{c}{Absolute} & \multicolumn{3}{c}{Relative} \\
 \cmidrule(lr){3-5} 
 \cmidrule(lr){6-8} 
 Decoder & Encoder & Acc & FScore & MAE & Acc & FScore & MAE \\
\midrule
\multirow{2}{*}{en} & en &  $60.48 \pm 1.02$ &  $60.62 \pm 0.89$ &  $45.30 \pm 1.07$ &  $60.48 \pm 1.02$ &  $60.62 \pm 0.89$ &  $45.30 \pm 1.07$ \\
   & fr &  $51.23 \pm 0.62$ &  $51.57 \pm 0.47$ &  $57.24 \pm 0.44$ &  $51.23 \pm 0.62$ &  $51.57 \pm 0.47$ &  $57.24 \pm 0.44$ \\
&\\
\multirow{2}{*}{fr} & en &  $59.78 \pm 1.89$ &  $60.03 \pm 1.89$ &  $46.92 \pm 4.19$ &  $59.78 \pm 1.89$ &  $60.03 \pm 1.89$ &  $46.92 \pm 4.19$ \\
   & fr &  $50.79 \pm 0.70$ &  $51.17 \pm 0.69$ &  $57.56 \pm 0.24$ &  $50.79 \pm 0.70$ &  $51.17 \pm 0.69$ &  $57.56 \pm 0.24$ \\
\bottomrule
\end{tabular}
}
\caption{Fine grained: topo_redo}
\label{tab:multilingual-topo_redo-fine-grained}
\end{table}


Absolute                                 Relative        \
                     acc       fscore          mae            acc         
                    mean   std   mean   std   mean   std     mean   std   
Decoder Encoder                                                           
en      en         60.48  1.02  60.62  0.89  45.30  1.07    60.48  1.02   
        fr         51.23  0.62  51.57  0.47  57.24  0.44    51.23  0.62   
fr      en         59.78  1.89  60.03  1.89  46.92  4.19    59.78  1.89   
        fr         50.79  0.70  51.17  0.69  57.56  0.24    50.79  0.70   

                                           
                fscore          mae        
                  mean   std   mean   std  
Decoder Encoder                            
en      en       60.62  0.89  45.30  1.07  
        fr       51.57  0.47  57.24  0.44  
fr      en       60.03  1.89  46.92  4.19  
        fr       51.17  0.69  57.56  0.24